In [47]:
#-------------------------------
#Funktionen:
#-------------------------------

# Print iterations progress
def printProgressBar (iteration, total, prefix = '', suffix = '', decimals = 1, length = 100, fill = '█', printEnd = "\r"):
    """
    Call in a loop to create terminal progress bar
    @params:
        iteration   - Required  : current iteration (Int)
        total       - Required  : total iterations (Int)
        prefix      - Optional  : prefix string (Str)
        suffix      - Optional  : suffix string (Str)
        decimals    - Optional  : positive number of decimals in percent complete (Int)
        length      - Optional  : character length of bar (Int)
        fill        - Optional  : bar fill character (Str)
        printEnd    - Optional  : end character (e.g. "\r", "\r\n") (Str)
    """
    percent = ("{0:." + str(decimals) + "f}").format(100 * (iteration / float(total)))
    filledLength = int(length * iteration // total)
    bar = fill * filledLength + '-' * (length - filledLength)
    print('\r%s |%s| %s%% %s' % (prefix, bar, percent, suffix), end = printEnd)
    # Print New Line on Complete
    if iteration == total: 
        print()

In [48]:
import numpy as np
from osgeo import gdal
from osgeo import osr
import time

#-------------------------------
#File einlesen:
#-------------------------------
fobj = open('Daten/Waldpunktwolke_1000.txt', "r") #Input File

pointlist = []

next(fobj)
for line in fobj:
    line = line.strip()
    line= line.split("\t")   
    pointlist.append(line)
fobj.close

<function TextIOWrapper.close()>

In [49]:
#-------------------------------
#Daten einlesen und nach Typ Filtern:
#-------------------------------

pointarray = np.array(pointlist) 
pointarray = pointarray.astype(float)  
print(len(pointarray))

#Neuer Array ohne Gebauede und Boden, Klassen 3, 4, 5
vegetationslist = []
bodenlist=[]

print("Vegetations-und Bodenliste einlesen:")
printProgressBar(0, len(pointarray), prefix = 'Progress:', suffix = 'Complete', length = 50)
j=0
for i in pointarray:
    printProgressBar(j+1, len(pointarray), prefix = 'Progress:', suffix = 'Complete', length = 50)
    j+=1
    if i[6] == 3 or i[6] ==4 or i[6] ==5:
        vegetationslist.append(i)
    if i[6] == 2:
        bodenlist.append(i)

vegarray = np.array(vegetationslist)        
bodenarray = np.array(bodenlist)


#----------------------------------
#Extend Array with random numbers:
#----------------------------------
#max values
maxvals = np.amax(pointarray,axis=0) 
xmax=maxvals[0]
ymax=maxvals[1]
# print ("xmax:",xmax)
# print ("ymax:",ymax)

#min values
minvals = np.amin(pointarray,axis=0) 
xmin=minvals[0]
ymin=minvals[1]
# print ("xmin:",xmin)
# print ("ymin:",ymin)

#empty array with extent dimensions
nrows= int(np.ceil(ymax-ymin))
ncols= int(np.ceil(xmax-xmin))

bodenout_array = np.empty((nrows,ncols))
bodenout_array.fill(0)

vegcountout_array = np.empty((nrows,ncols))
vegcountout_array.fill(0)

veghoeheout_array = np.empty((nrows,ncols))
veghoeheout_array.fill(0)

1000
Vegetations-und Bodenliste einlesen:
Progress: |██████████████████████████████████████████████████| 100.0% Complete


In [50]:
#----------------------------------
#Bodenpunkte zaehlen:
#----------------------------------
bodencount = 0
countlist =[]
arraycount_x = 0
arraycount_y = 0
bodenhoehenlist = []

print("Bodenpunkte zuordnen:")
printProgressBar(0, len(np.arange(ymax,ymax-nrows,-1)), prefix = 'Progress:', suffix = 'Complete', length = 50)
j=0
for y in np.arange(ymax,ymax-nrows,-1):
    printProgressBar(j+1, len(np.arange(ymax,ymax-nrows,-1)), prefix = 'Progress:', suffix = 'Complete', length = 50)
    j+=1
    # print("yRichtung:",y)
    # print("ArraypositionY:",arraycount_y)
    for x in np.arange(xmin,xmin+ncols,1):
        # print("xRichtung:",x)
        # print("ArraypositionX:",arraycount_x)
        for i in bodenarray:
            if x <= i[0] < x+1 and y >= i[1] > y-1:
                bodencount+=1
                hoehenlist.append(i[2])

        bodenout_array[arraycount_y][arraycount_x]=bodencount
        arraycount_x+=1     
        bodencount=0
    arraycount_x=0
    arraycount_y+=1
    # print("xRichtungEnde")    




Bodenpunkte zuordnen:
Progress: |██████████████████████████████████████████████████| 100.0% Complete


In [51]:
#----------------------------------
#Mittlere Hoehe der VegPunkte:
#----------------------------------
vegcount = 0
countlist =[]
arraycount_x = 0
arraycount_y = 0
veghoehenlist = []

print("VegPunkte zuordnen:")
printProgressBar(0, len(np.arange(ymax,ymax-nrows,-1)), prefix = 'Progress:', suffix = 'Complete', length = 50)
j=0
for y in np.arange(ymax,ymax-nrows,-1):
    printProgressBar(j+1, len(np.arange(ymax,ymax-nrows,-1)), prefix = 'Progress:', suffix = 'Complete', length = 50)
    j+=1
    # print("yRichtung:",y)
    # print("ArraypositionY:",arraycount_y)
    for x in np.arange(xmin,xmin+ncols,1):
        # print("xRichtung:",x)
        # print("ArraypositionX:",arraycount_x)
        for i in vegarray:
            if x <= i[0] < x+1 and y >= i[1] > y-1: #Hier sind wir in der jeweiligen Zelle
                veghoehenlist.append(i[2])
                
                vegcount+=1

        veghoeheout_array[arraycount_y][arraycount_x]=np.mean(veghoehenlist) #Mittlere Hoehe der VegPunkte
        vegcountout_array[arraycount_y][arraycount_x]=vegcount #Anzahl der Vegetationspunkte in Zelle
        arraycount_x+=1     
        vegcount=0
        hoehenlist=[]
    arraycount_x=0
    arraycount_y+=1
    # print("xRichtungEnde") 

VegPunkte zuordnen:
[]
[]
[972.63, 972.57, 972.44, 972.62, 973.26, 972.48, 972.95, 972.4, 972.51]
[972.6, 972.62, 972.64, 972.54, 972.42, 973.05, 972.8, 972.34]
[972.66, 972.69, 973.01, 972.55, 972.32, 972.49, 972.52, 972.56]
[972.12, 972.77, 972.56, 972.37]
[]
[972.0]
[976.22, 975.46, 975.51, 972.27, 972.07]
[977.05, 976.18, 974.76, 975.76, 975.5, 975.3, 974.72, 974.83, 974.87, 973.53, 973.08, 972.83, 976.19, 976.06, 975.22, 974.96, 972.23, 972.13, 972.07]
[974.59, 974.36, 973.92, 972.27, 972.05, 975.63, 975.06, 974.79, 974.77, 972.08, 971.89, 976.25, 975.58, 974.75, 974.98, 973.24, 972.07, 972.02, 971.91]
[974.34, 974.49, 973.6, 975.35, 974.51, 974.0, 974.22, 972.3, 971.9, 971.69, 971.87, 971.87, 971.69, 971.82]
[]
[]
[972.39, 972.36, 972.42, 972.3, 972.34, 972.21]
[972.17]
[972.0, 972.32, 972.38, 972.23]
[971.89]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[972.36]
[]
[]
[]
[]
[971.4]
[]
[]
[]
[]
[]
[972.19, 971.9, 971.77, 971.76]
[972.3, 972.19, 972.2, 972.

In [52]:
#-------------------------------
#OutRaster schreiben:
#-------------------------------
#Boden
driver = gdal.GetDriverByName("GTiff")
dataset = driver.Create("Bodenpunkte_1000.tif", ncols, nrows, 1, gdal.GDT_Float32)
dataset.SetGeoTransform((xmin,1,0,ymax,0,-1))

#Koordinatensystem definieren:
dstSRS = osr.SpatialReference()
dstSRS.ImportFromEPSG(32632)
dest_wkt = dstSRS.ExportToWkt()

dataset.SetProjection(dest_wkt)

#Raster ausgeben:
bandout = dataset.GetRasterBand(1).WriteArray(bodenout_array)
dataset.FlushCache()

In [53]:

#Vegetation Zaehler
driver = gdal.GetDriverByName("GTiff")
dataset = driver.Create("VegCountPunkte_1000.tif", ncols, nrows, 1, gdal.GDT_Float32)
dataset.SetGeoTransform((xmin,1,0,ymax,0,-1))

#Koordinatensystem definieren:
dstSRS = osr.SpatialReference()
dstSRS.ImportFromEPSG(32632)
dest_wkt = dstSRS.ExportToWkt()

dataset.SetProjection(dest_wkt)

#Raster ausgeben:
bandout = dataset.GetRasterBand(1).WriteArray(vegcountout_array)
dataset.FlushCache()

In [54]:

#Vegetation Mittlere Hoehe Raster
driver = gdal.GetDriverByName("GTiff")
dataset = driver.Create("VegHoehenPunkte_1000.tif", ncols, nrows, 1, gdal.GDT_Float32)
dataset.SetGeoTransform((xmin,1,0,ymax,0,-1))

#Koordinatensystem definieren:
dstSRS = osr.SpatialReference()
dstSRS.ImportFromEPSG(32632)
dest_wkt = dstSRS.ExportToWkt()

dataset.SetProjection(dest_wkt)

#Raster ausgeben:
bandout = dataset.GetRasterBand(1).WriteArray(veghoeheout_array)
dataset.FlushCache()